In [1]:
%%capture
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd
import re
import csv
from tqdm import tqdm
import gc

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-Instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


==((====))==  Unsloth 2024.10.7: Fast Qwen2 patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/139 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


# Load the dataset



In [5]:
import pandas as pd
df=pd.read_csv("/kaggle/input/dlsprint3/train.csv")

In [6]:
prompt = """Your task is to solve a math problem given in Bengali. Follow these steps:

1.Translate the problem into English.
2.Identify the key information and variables.
3.Outline your approach to solving the problem.
4.Show your work step by step, explaining each step clearly.
5.Write your final answer within \boxed{{}} notation.
6.Provide a brief explanation of the result.

Ensure your solution is clear and easy to follow. If there are multiple possible answers, mention this and explain why.
Problem:
{}
Solution:
{}
"""

EOS_TOKEN = tokenizer.eos_token
 # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Problem"]
    outputs      = examples["Answer"]
    texts = []
    for  input, output in zip( inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [7]:
tokenizer.padding_side = 'right'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
#     eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 0,
#         max_steps = 10,
        num_train_epochs = 1,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
#         load_best_model_at_end = True,
#         save_strategy = 'steps',
#         eval_strategy = 'steps',
#         save_total_limit = 1,
#         metric_for_best_model = 'loss'
    ),
)

Map (num_proc=2):   0%|          | 0/209 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 209 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 26
 "-____-"     Number of trainable parameters = 40,370,176


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


Step,Training Loss
10,2.639200
20,2.216300


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [10]:
import re

In [11]:
def extract_answer(result):
    match = re.search(r'\\boxed{(.*?)}', result)
    if match:
        boxed_content = match.group(1)
        digits = ''.join(filter(str.isdigit, boxed_content))
        if digits:
            return int(digits)
    return None

In [12]:
def majority_answer(answers):
    answers = [answer for answer in answers if answer is not None]

    if not answers:
        return None
    
    counts = {}
    for answer in answers:
        if answer in counts:
            counts[answer] += 1
        else:
            counts[answer] = 1

    max_answer = None
    max_count = 0
    
    for answer, count in counts.items():
        if count > max_count:
            max_answer = answer
            max_count = count
    
    return max_answer

In [13]:
test_df=pd.read_csv("/kaggle/input/dlsprint3/test.csv")

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def generate_response(input_text):
    prompt_1 = prompt.format(input_text,"")
    inputs = tokenizer([prompt_1], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=2000, use_cache=True,temperature=1, min_p=0.1)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # Extract the part where SUBJ or OBJ is written
#     response_text = response[0].split("### Output:\n")[1].strip().split('\n')[0]
#     if(len(response_text)>1):
#         response_text=0
    
    answers = [extract_answer(result) for result in response]
    
#     print(majority_answer(answers))
    
    return majority_answer(answers)

In [15]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [16]:
file = open('submission.csv', 'w', encoding='utf-8')
writer = csv.writer(file)
writer.writerow(['ID', 'Answer'])

for row in tqdm(test_df.values):
    id = row[0]
    problem = row[1]    
    answer = generate_response(problem)
    print(answer)
    
    
    if answer is None:
        answer = 36
    print(answer)
        
    writer.writerow([id, answer])
    
file.close()

  1%|          | 1/100 [00:31<52:17, 31.69s/it]

4
4


  2%|▏         | 2/100 [01:11<59:31, 36.45s/it]

10
10


  3%|▎         | 3/100 [03:22<2:08:53, 79.72s/it]

...

521
521


 99%|█████████▉| 99/100 [1:38:25<00:55, 55.36s/it]

71
71


100%|██████████| 100/100 [1:38:56<00:00, 59.37s/it]

4
4
